In [1]:
from src.datasets import stellar_data
from src.models import vanilla_stellar
import torch
import argparse

d:\dlls-pro\dlls-pro-venv\Lib\site-packages\torch_geometric\typing.py:110: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: [WinError 127] Nie można odnaleźć określonej procedury
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


In [2]:
torch.manual_seed(42)
sample_idx_permutations = []
n_folds = 5
n_train = 100
for i in range(n_folds):
    idx_perm = torch.randperm(125).tolist()
    sample_idx_permutations.append({"train" : idx_perm[:n_train], "valid" : idx_perm[n_train:]})

In [3]:
k = 0
train_dataloader = stellar_data.StellarDataloader(
    filename='stellar_graph_dataset_train.pt',
    batch_size=512,
    shuffle=True,
    graphs_idx=sample_idx_permutations[k]["train"],
    test=False
)
valid_dataloader = stellar_data.StellarDataloader(
    filename='stellar_graph_dataset_train.pt',
    batch_size=512,
    shuffle=False,
    graphs_idx=sample_idx_permutations[k]["valid"],
    test=False
)

In [4]:
cfg_reduced = argparse.Namespace(**{
    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    'input_dim': 41,
    'hid_dim': 128,
    'num_classes': 14,
    'lr': 1e-2,
})

vanilla_stellar_reduced = vanilla_stellar.VanillaStellarReduced(cfg_reduced)

In [5]:
vanilla_stellar_reduced.train(
    train_loader=train_dataloader,
    valid_loader=valid_dataloader,
    epochs=20)

Validation - epoch 19:   4%|▍         | 1/25 [00:00<00:02,  8.40it/s, Loss=1.65, Accuracy=0.55]
